# NN - PHYS 555 Term Project - Sam and Breanna

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'Running on : {device}')

Running on : cpu


In [3]:
# data load
X = np.load("Data/Input_Class_3Classes_Sep.npy") # Load input data
Y = np.load("Data/Target_Class_3Classes_Sep.npy") # Load target data

# splitting data
inp_tr, inp_va, tar_tr, tar_va = train_test_split(X, Y, test_size=0.25)

print(f'Training set: {inp_tr.shape} , {tar_tr.shape} ------- Validation set: {inp_va.shape} , {tar_va.shape}')

# scaling data according to training inputs
scaler_S = StandardScaler().fit(inp_tr)
inp_tr = scaler_S.transform(inp_tr)
inp_va = scaler_S.transform(inp_va)

Training set: (20177, 8) , (20177, 1) ------- Validation set: (6726, 8) , (6726, 1)


In [6]:
# checking labels
#plt.hist(tar_tr)

ind = len(np.where(tar_tr == 2)[0])
print(ind/len(tar_tr))

0.8854140853446995


In [231]:
# concatenate the labels onto the inputs for both training and validation
inp_tr = torch.tensor(inp_tr)
tar_tr = torch.tensor(tar_tr)
inp_va = torch.tensor(inp_va)
tar_va = torch.tensor(tar_va)

train_data = data_utils.TensorDataset(inp_tr, tar_tr)
test_data = data_utils.TensorDataset(inp_va, tar_va)

# constructing data loaders for nn
train_loader = torch.utils.data.DataLoader(train_data, batch_size=50)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=50)

/opt/anaconda3/envs/PHYS555_PY37/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/opt/anaconda3/envs/PHYS555_PY37/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/envs/PHYS555_PY37/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.
/opt/anaconda3/envs/PHYS555_P

In [232]:
# tester code to see if loaders are behaving properly
for i in range(1):
    item = train_loader.dataset.__getitem__(i)
    print(item)

(tensor([-0.1199, -0.4571, -0.0471, -0.6029, -0.0378, -0.7544,  0.1209, -0.8241],
       dtype=torch.float64), tensor([2], dtype=torch.int32))


## Creating Replicate NN from Cornu Paper

> What do they use?

Sigmoid Activation function for the layer(s)
Sum of squares difference for loss function
Only one hidden layer
Output layer must have 3 neurons to match classes amount
Output layer has the softmax activation
Input layer has a dimension of 8


In [151]:
# NN - With one hidden layer
class BaseMLP(nn.Module):
    def __init__(self, input_size, n_hidden, output_size):
        super(BaseMLP, self).__init__()
        self.input_size = input_size
        self.n_hidden = n_hidden
        self.output_size = output_size
        self.fc1 = nn.Linear(self.input_size, self.n_hidden)
        self.fc2 = nn.Linear(self.n_hidden, self.output_size)
        
    def forward(self, x):
        x = self.fc1(x)
        x = torch.sigmoid(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x

In [233]:
def get_n_params(model):
    np=0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

learning_rate = 0.005
mom = 0.9

accuracy_list = []

def train(epoch, model):
    model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=mom)
    print('Number of parameters: {}'.format(get_n_params(model)))

    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # send to device
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data.float())
        loss = F.cross_entropy(output, target.squeeze(-1).long())
        loss.backward()
        optimizer.step()
        # Print out every 10 batches
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} \
({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
            
def test(model):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        # send to device
        data, target = data.to(device), target.to(device)
        
        output = model(data.float())
        test_loss += F.cross_entropy(output, target.squeeze(-1).long(), reduction='sum').item() # sum up batch loss                                                               
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability                                                                 
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    accuracy_list.append(accuracy)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: \
    {correct}/{len(test_loader.dataset)} ({accuracy:.0f}%)\n')

In [234]:
# create nn instance
BaseNN = BaseMLP(8, 20, 3)

epochs = 100

for epoch in range(0, epochs):
    train(epoch, BaseNN)
    test(BaseNN)

Number of parameters: 243
Train Epoch: 0 [0/20177 (0%)]	Loss: 1.174527
Train Epoch: 0 [5000/20177 (25%)]	Loss: 0.675737
Train Epoch: 0 [10000/20177 (50%)]	Loss: 0.739549
Train Epoch: 0 [15000/20177 (74%)]	Loss: 0.678168
Train Epoch: 0 [20000/20177 (99%)]	Loss: 0.657039

Test set: Average loss: 0.6723, Accuracy:     5948/6726 (88%)

Number of parameters: 243
Train Epoch: 1 [0/20177 (0%)]	Loss: 0.617834
Train Epoch: 1 [5000/20177 (25%)]	Loss: 0.655808
Train Epoch: 1 [10000/20177 (50%)]	Loss: 0.733940
Train Epoch: 1 [15000/20177 (74%)]	Loss: 0.674234
Train Epoch: 1 [20000/20177 (99%)]	Loss: 0.654233

Test set: Average loss: 0.6697, Accuracy:     5948/6726 (88%)

Number of parameters: 243
Train Epoch: 2 [0/20177 (0%)]	Loss: 0.614678
Train Epoch: 2 [5000/20177 (25%)]	Loss: 0.653828
Train Epoch: 2 [10000/20177 (50%)]	Loss: 0.732877
Train Epoch: 2 [15000/20177 (74%)]	Loss: 0.673183
Train Epoch: 2 [20000/20177 (99%)]	Loss: 0.653299

Test set: Average loss: 0.6688, Accuracy:     5948/6726 (88%)